# 🎌 MyAnimeList Orderer

> **📚✨ Discover What to Watch or Read Next — Effortlessly! 🎥🌟**

Are you overwhelmed by your ever-growing anime and manga backlog on **MyAnimeList**?
Let this tool do the thinking for you! 🧠💡

With just a few clicks, you can:
- 🎯 **Sort** your list using smart filters and custom criteria
- 📊 **Enrich** entries with ratings, rankings, and descriptions
- 🌐 **Find** where to stream titles in your country via **JustWatch**
- 💾 **Export** a clean, organized table to use or share

No more endless scrolling — just clear, curated recommendations tailored to your preferences.
Let’s make your next pick the right one! ✅



## 📋 What Does This Notebook Do?

This Jupyter notebook is designed to streamline your anime or manga decision-making process using data from **MyAnimeList** and **JustWatch**.

---

### 🔍 How It Works

1. **Select a List Section**
   You choose which part of your MyAnimeList to work with (e.g., "Plan to Watch", "Watching", "Completed").

2. **Sort by Your Preferred Criteria**
   The notebook organizes your anime or manga based on the sorting criteria you specify (e.g., score, popularity, number of episodes, etc.).

3. **Data Enrichment via MyAnimeList API**
   After sorting, the notebook queries the **MyAnimeList API v2** to enrich each entry with additional details such as:
   - ✅ Title and description
   - 🎬 Number of episodes or chapters
   - 🔁 Whether it has sequels/prequels
   - ⭐ User score and global ranking

   📚 **API Reference**: [MyAnimeList API v2 Documentation](https://myanimelist.net/apiconfig/references/api/v2#section/)

4. **Streaming Availability via JustWatch**
   The notebook uses the **Simple JustWatch Python API** to fetch streaming information, showing where each title is available **in your country**.

   📚 **Library Reference**: [simple-justwatch-python-api on PyPI](https://pypi.org/project/simple-justwatch-python-api/)

---

### 📊 Output

You’ll get a clean, well-organized table with all this information, making it easier to choose what to watch or read next.


## 📦 Installing and Importing Dependencies

By running the following cell, **all required libraries will be installed and imported automatically**.
This ensures the notebook is ready to use without any manual setup.

If you encounter any errors during the process, please install the dependencies manually using `pip`.

### 📋 Required Dependencies

- `os` *(standard library, no installation needed)*
- `pandas`
- `requests`
- `simple-justwatch-python-api`
- `secrets` *(standard library, no installation needed)*
- `base64` *(standard library, no installation needed)*
- `urllib.parse` *(standard library, no installation needed)*
- `webbrowser` *(standard library, no installation needed)*
- `re` *(standard library, no installation needed)*
- `IPython.display` *(standard library, no installation needed)*
- `warnings` *(standard library, no installation needed)*

In [21]:
#First, we need to import/install the required libraries
import os
!pip install pandas
import pandas as pd
!pip install requests
import requests
!pip install simple-justwatch-python-api
from simplejustwatchapi.justwatch import search, details
import secrets
import base64
import urllib.parse
import webbrowser
import re
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

## 🔑 Setting Up Your MyAnimeList API Key

To access the MyAnimeList API, you need a **Client ID** (API key).
In this notebook, we load it from your system's **environment variables** to keep your credentials secure.

### 📥 How to Get Your API Key

1. Go to the official MyAnimeList developer portal.
2. Create an application.
3. Copy your **Client ID**.

📚 **Documentation**: [MyAnimeList API v2](https://myanimelist.net/apiconfig/references/api/v2#section/)

---

### ⚙️ Load the API Key in Python

Make sure you've saved your `MAL_CLIENT_ID` as an environment variable.



In [22]:
import os

# Load the MyAnimeList Client ID from environment variables
MAL_API_KEY = os.getenv("MAL_API_KEY").strip()

# Optional: show error if not set
if not MAL_API_KEY:
    raise ValueError("❌ MAL_CLIENT_ID not found. Please set it as an environment variable.")


## 🌍 Setting the Search Country

To check streaming availability using **JustWatch**, we need to specify the country where the search will be performed.

This is done by setting a constant with the [two-letter country code (ISO 3166-1 alpha-2)](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes).
For example:
- `US` → United States
- `ES` → Spain
- `JP` → Japan
- `BR` → Brazil

### ✏️ Set Your Country Code



In [23]:
# Set your country code for JustWatch
# Example: 'ES' for Spain
COUNTRY = "ES"

## 🔐 OAuth2 Authentication with MyAnimeList

To access your personal anime and manga lists from the **MyAnimeList API**, we need to authenticate using the **OAuth2** protocol.

This notebook automates the process and guides you through it step by step.

---

### 🧩 What Happens Behind the Scenes?

1. **PKCE Generation**
   A secure code verifier and challenge are generated as part of the OAuth2 flow (using the [PKCE](https://datatracker.ietf.org/doc/html/rfc7636) method).
   *Note: MyAnimeList only supports the `plain` method for PKCE.*

2. **Authorization URL**
   A browser window opens prompting you to authorize the application.
   After accepting, you’ll be redirected to a `404` error page — this is expected.

3. **Code Extraction**
   You’ll copy the full URL from that `404` page and paste it into the notebook.
   The notebook will automatically extract the **authorization code** from it.

4. **Token Exchange**
   That code is securely sent to the MyAnimeList servers to obtain an **access token**.

5. **Token Validation**
   The access token is tested by fetching your profile data from MyAnimeList to confirm authentication succeeded.

6. **Environment Setup**
   If all goes well, your access token is saved in the `OAuth2` variable and stored as an environment variable to simplify future API calls.

---

### ✅ What You Need to Do

- Make sure your `MAL_API_KEY` (Client ID) is already loaded in the environment.
- When prompted:
  - Approve the application in the browser.
  - Copy the **entire URL** you are redirected to (even if it shows a 404).
  - Paste it into the notebook when asked.

> If successful, your authentication will be saved and ready to use across the rest of the notebook.

---


In [24]:
def generate_pkce():
    """
    Generates code_verifier and code_challenge for PKCE
    MAL only supports 'plain' method
    """
    code_verifier = base64.urlsafe_b64encode(secrets.token_bytes(96)).decode('utf-8')
    code_verifier = code_verifier.rstrip('=')

    # For MAL: code_challenge = code_verifier (plain method)
    code_challenge = code_verifier

    return code_verifier, code_challenge

def get_access_token_manual(client_id, authorization_code, code_verifier):
    """
    Exchanges authorization code for access token
    """
    url = "https://myanimelist.net/v1/oauth2/token"

    # Form data (WITHOUT redirect_uri according to Stack Overflow)
    data = {
        "client_id": client_id,
        "client_secret": "",  # Empty for applications without client_secret
        "grant_type": "authorization_code",
        "code": authorization_code,
        "code_verifier": code_verifier
    }

    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    print("🔄 Exchanging code for access token...")
    response = requests.post(url, data=data, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None

def extract_code_from_url(full_url):
    """
    Extracts authorization code from the complete URL
    """
    try:
        # Search for 'code' parameter in URL
        match = re.search(r'code=([^&]+)', full_url)
        if match:
            return match.group(1)
        else:
            return None
    except:
        return None

def oauth2_automatic():
    """
    Automatic OAuth2 process using MAL_API_KEY
    """
    print("🚀 Starting automatic OAuth2 for MyAnimeList...")

    # Check if MAL_API_KEY exists
    try:
        CLIENT_ID = MAL_API_KEY.strip()
        if not CLIENT_ID:
            raise ValueError("MAL_API_KEY is empty")
    except NameError:
        print("❌ ERROR: MAL_API_KEY variable not found")
        print("💡 Define MAL_API_KEY in a previous cell with your Client ID")
        return None
    except Exception as e:
        print(f"❌ ERROR: Problem with MAL_API_KEY: {e}")
        return None

    print(f"🔑 Using Client ID: {CLIENT_ID}")

    # Generate PKCE
    code_verifier, code_challenge = generate_pkce()
    print(f"📝 Code verifier generated: {code_verifier[:20]}...")

    # Build authorization URL (WITHOUT redirect_uri according to Stack Overflow)
    params = {
        "response_type": "code",
        "client_id": CLIENT_ID,
        "code_challenge": code_challenge,
        "code_challenge_method": "plain",
        "state": secrets.token_urlsafe(32)
    }

    base_url = "https://myanimelist.net/v1/oauth2/authorize"
    query_string = urllib.parse.urlencode(params)
    authorization_url = f"{base_url}?{query_string}"

    print("\n" + "="*70)
    print("📋 INSTRUCTIONS TO GET THE CODE:")
    print("1. Your browser will open automatically")
    print("2. Authorize the application on MyAnimeList")
    print("3. You'll be redirected to a 404 error page - THIS IS NORMAL!")
    print("4. Copy the COMPLETE URL from the address bar")
    print("5. Paste it when requested (code will be extracted automatically)")
    print("="*70)

    print(f"\n🔗 Authorization URL:")
    print(authorization_url)

    # Open browser automatically
    try:
        webbrowser.open(authorization_url)
        print("\n🌐 Browser opened automatically")
    except:
        print("\n⚠️  Could not open browser automatically")

    # Request complete URL
    while True:
        print("\n📥 After authorizing, you'll see a 404 error page.")
        print("📥 Copy the COMPLETE URL from the address bar and paste it here:")
        full_url = input("Complete URL: ").strip()

        if not full_url:
            print("❌ Empty URL. Try again.")
            continue

        # Extract code automatically
        authorization_code = extract_code_from_url(full_url)

        if not authorization_code:
            print("❌ Could not extract code from URL.")
            print("💡 Make sure to copy the complete URL that appears after authorizing.")
            continue

        if len(authorization_code) < 10:
            print("❌ Extracted code too short. Check the URL.")
            continue

        print(f"✅ Code extracted automatically: {authorization_code[:20]}...")
        break

    # Exchange for access token
    token_data = get_access_token_manual(CLIENT_ID, authorization_code, code_verifier)

    return token_data

def test_access_token_automatic(access_token):
    """
    Automatic access token test
    """
    print("\n🧪 Testing access token...")

    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Test with user information endpoint
    url = "https://api.myanimelist.net/v2/users/@me"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        user_data = response.json()
        print("✅ ACCESS TOKEN VALID!")
        print(f"👤 Authenticated user: {user_data.get('name', 'N/A')}")
        print(f"🆔 User ID: {user_data.get('id', 'N/A')}")
        return True
    else:
        print(f"❌ ACCESS TOKEN INVALID!")
        print(f"Error {response.status_code}: {response.text}")
        return False

# AUTOMATIC EXECUTION
print("🎌 MyAnimeList OAuth2 - Automatic Execution")
print("="*50)

# Initialize OAuth2 variable
OAuth2 = None

try:
    # Execute OAuth2 process
    tokens = oauth2_automatic()

    if tokens:
        print("\n🎉 TOKENS OBTAINED SUCCESSFULLY!")
        print(f"🔐 Access Token: {tokens['access_token'][:50]}...")
        print(f"🔄 Refresh Token: {tokens['refresh_token'][:50]}...")
        print(f"⏰ Expires in: {tokens['expires_in']} seconds")

        # Test automatically
        if test_access_token_automatic(tokens['access_token']):
            # Store in OAuth2 variable
            OAuth2 = {
                'access_token': tokens['access_token'],
                'refresh_token': tokens['refresh_token'],
                'expires_in': tokens['expires_in'],
                'token_type': tokens.get('token_type', 'Bearer'),
                'status': 'valid'
            }

            # Also save in environment variables
            os.environ['MAL_ACCESS_TOKEN'] = tokens['access_token']
            os.environ['MAL_REFRESH_TOKEN'] = tokens['refresh_token']

            print("\n🎊 SETUP COMPLETED SUCCESSFULLY!")
            print("✅ Tokens stored in 'OAuth2' variable")
            print("✅ Environment variables configured")
            print("✅ Authentication verified")
            print("\n💡 Now you can use OAuth2['access_token'] to make requests")

        else:
            OAuth2 = {
                'status': 'error',
                'message': 'Token obtained but not valid'
            }
            print("\n❌ SETUP FAILED!")
            print("❌ Tokens were obtained but are not valid")

    else:
        OAuth2 = {
            'status': 'error',
            'message': 'Could not obtain tokens'
        }
        print("\n❌ SETUP FAILED!")
        print("❌ Could not obtain OAuth2 tokens")

except Exception as e:
    OAuth2 = {
        'status': 'error',
        'message': f'Error during process: {str(e)}'
    }
    print(f"\n💥 CRITICAL ERROR: {e}")
    print("❌ SETUP FAILED!")

# Show final status
print("\n" + "="*50)
print("📊 FINAL STATUS:")
if OAuth2 and OAuth2.get('status') == 'valid':
    print("🟢 OAuth2 configured correctly")
    print(f"🔑 Access token available: OAuth2['access_token']")
else:
    print("🔴 OAuth2 not configured")
    if OAuth2:
        print(f"❌ Reason: {OAuth2.get('message', 'Unknown error')}")
print("="*50)


🎌 MyAnimeList OAuth2 - Automatic Execution
🚀 Starting automatic OAuth2 for MyAnimeList...
🔑 Using Client ID: b5aa38ba313fd306c2804acf13e4a5f0
📝 Code verifier generated: xv9zwnHw4a3XqJdEBIoY...

📋 INSTRUCTIONS TO GET THE CODE:
1. Your browser will open automatically
2. Authorize the application on MyAnimeList
3. You'll be redirected to a 404 error page - THIS IS NORMAL!
4. Copy the COMPLETE URL from the address bar
5. Paste it when requested (code will be extracted automatically)

🔗 Authorization URL:
https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=b5aa38ba313fd306c2804acf13e4a5f0&code_challenge=xv9zwnHw4a3XqJdEBIoYXtC0IMSeiS3M6tCMzJuFKpbgHFt2be_HZrMtniyqFjve6QmmMSuGI6YhNiselbagiEG_9VBk7iP8HFcvqpyhjN5-JYLehwONvqQyQVlLnbPc&code_challenge_method=plain&state=jmIEqP3qoJl8_9YzeM-pWT95UQ35c9TPY_ornSj8b8M

🌐 Browser opened automatically

📥 After authorizing, you'll see a 404 error page.
📥 Copy the COMPLETE URL from the address bar and paste it here:
✅ Code extracted au

## 🎬 MyAnimeList Table Generator with Streaming Platforms

---

### 🧩 Core Functionality

1. **Fetch User Anime List**
   - Retrieves your anime list from MyAnimeList based on the selected status (e.g., watching, completed, plan to watch).
   - Uses individual API requests to gather comprehensive data for each anime entry.

2. **Detailed Anime Data Retrieval**
   - For each anime in your list, fetches detailed information from MyAnimeList including:
     - ✅ Main title and alternative titles (English, Japanese, synonyms)
     - ⭐ Your personal score and MyAnimeList community score
     - 📺 Episode progress and total episodes
     - 🏆 Global ranking and popularity metrics
     - 🎬 Cover images and metadata

3. **Streaming Availability Check**
   - Uses the **Simple JustWatch Python API** to check where each anime is available for streaming in your specified country.
   - Searches using multiple title variations to maximize match accuracy.
   - Provides real-time availability across major streaming platforms.

4. **Data Processing and Sorting**
   - Processes and sorts the anime data based on user-selected criteria such as:
     - Personal scores, MAL community scores, global rankings
     - Episode progress, completion percentage
     - Alphabetical title sorting
   - Handles special cases like unranked content and unscored entries.

5. **Interactive Table Generation**
   - Creates a clean, styled HTML table displaying:
     - Order ranking, anime titles with cover images
     - Selected sorting parameter and additional rating information
     - **Streaming platform availability** in the dedicated column
   - Optimized for Jupyter notebook environments with responsive design.



### 🌍 Streaming Platform Integration

The system automatically checks streaming availability by:
- Searching JustWatch using the anime's main title and alternative titles
- Filtering results based on your specified country code
- Displaying platform names where the anime is currently available
- Providing clear status messages for unavailable or unfound content

---

### 🎯 Why This Approach?

This implementation prioritizes **data accuracy** and **comprehensive coverage** by making individual API requests for each anime, ensuring you get the most up-to-date information about both your personal viewing progress and current streaming availability.



---

*Detailed usage instructions and setup requirements will be provided in the following cells.*


In [58]:
def get_streaming_platforms_for_anime(anime_details, country):
    """
    Gets streaming platforms for an anime using JustWatch API with multiple title attempts.

    Args:
        anime_details (dict): Anime details from MyAnimeList API
        country (str): Country code (e.g., 'US', 'ES', 'GB')

    Returns:
        str: Comma-separated list of streaming platforms or status message
    """
    try:
        titles_to_search = []

        # Main title
        main_title = anime_details.get('title', '')
        if main_title and len(main_title.strip()) > 2:
            titles_to_search.append(main_title.strip())

        # Alternative titles
        alternative_titles = anime_details.get('alternative_titles', {})
        if isinstance(alternative_titles, dict):
            # English titles
            if 'en' in alternative_titles and isinstance(alternative_titles['en'], list):
                for title in alternative_titles['en']:
                    if title and len(title.strip()) > 2:
                        titles_to_search.append(title.strip())

            # Japanese titles
            if 'ja' in alternative_titles and isinstance(alternative_titles['ja'], list):
                for title in alternative_titles['ja']:
                    if title and len(title.strip()) > 1:
                        titles_to_search.append(title.strip())

            # Synonyms
            if 'synonyms' in alternative_titles and isinstance(alternative_titles['synonyms'], list):
                for title in alternative_titles['synonyms']:
                    if title and len(title.strip()) > 2:
                        titles_to_search.append(title.strip())

        # Remove duplicates and filter valid titles
        titles_to_search = list(set([title for title in titles_to_search if title and len(title.strip()) > 1]))

        if not titles_to_search:
            return "No valid titles to search"

        # Try each title until we find a match
        for title in titles_to_search[:3]:  # Limit to first 3 titles
            try:
                search_results = search(title, country.upper(), "en", 3, True)
                if search_results and len(search_results) > 0:
                    best_match = search_results[0]

                    # Check if the entry has offers directly
                    if hasattr(best_match, 'offers') and best_match.offers:
                        platforms = []
                        for offer in best_match.offers:
                            if hasattr(offer, 'package') and hasattr(offer.package, 'name'):
                                platform_name = offer.package.name
                                if platform_name not in platforms:
                                    platforms.append(platform_name)

                        if platforms:
                            return ", ".join(platforms)
                    continue
            except Exception:
                continue

        return "Not available in this country"
    except Exception:
        return "Error checking availability"


def get_anime_list_ids(status):
    """
    Gets anime IDs from MyAnimeList by status using OAuth2.

    Args:
        status (str): Anime list status ('watching', 'completed', etc.)

    Returns:
        tuple: (list of anime IDs, error message)
    """
    try:
        oauth2_token = tokens['access_token']
    except (NameError, KeyError):
        return None, "OAuth2 tokens not available. Please authenticate first."

    headers = {"Authorization": f"Bearer {oauth2_token}"}
    params = {"fields": "anime_id", "limit": 1000, "status": status}
    url = "https://api.myanimelist.net/v2/users/@me/animelist"

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            anime_ids = [entry['node']['id'] for entry in data.get('data', [])]
            return anime_ids, None
        else:
            return None, f"Error {response.status_code}: {response.text}"
    except Exception as e:
        return None, f"Request failed: {str(e)}"


def get_individual_anime_details(anime_id):
    """
    Gets detailed information for a single anime.

    Args:
        anime_id (int): MyAnimeList anime ID

    Returns:
        dict: Anime details or None if error
    """
    try:
        oauth2_token = tokens['access_token']
    except (NameError, KeyError):
        return None

    headers = {"Authorization": f"Bearer {oauth2_token}"}
    params = {
        "fields": "id,title,alternative_titles,mean,num_episodes,rank,"
                 "main_picture{medium,large},genres,status,media_type,"
                 "start_date,end_date,synopsis,rating,source,studios"
    }
    url = f"https://api.myanimelist.net/v2/anime/{anime_id}"

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        return None
    except Exception:
        return None


def get_user_anime_status(anime_id, status):
    """
    Gets user's specific status information for an anime.

    Args:
        anime_id (int): MyAnimeList anime ID
        status (str): Anime list status

    Returns:
        dict: User's anime status information
    """
    try:
        oauth2_token = tokens['access_token']
    except (NameError, KeyError):
        return {}

    headers = {"Authorization": f"Bearer {oauth2_token}"}
    params = {
        "fields": "list_status{status,score,num_episodes_watched,is_rewatching,updated_at,start_date,finish_date}",
        "limit": 1000,
        "status": status
    }
    url = "https://api.myanimelist.net/v2/users/@me/animelist"

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            for entry in data.get('data', []):
                if entry['node']['id'] == anime_id:
                    return entry.get('list_status', {})
        return {}
    except:
        return {}


def process_individual_anime(anime_details, user_status):
    """
    Processes individual anime data combining API details and user status.

    Args:
        anime_details (dict): Anime details from MyAnimeList API
        user_status (dict): User's status for this anime

    Returns:
        dict: Processed anime data
    """
    try:
        # Get country from COUNTRY variable
        try:
            country = COUNTRY
        except NameError:
            country = None

        # Extract image URL
        main_picture = anime_details.get('main_picture', {})
        image_url = ""
        if isinstance(main_picture, dict):
            image_url = main_picture.get('medium', main_picture.get('large', ''))

        # Extract scores
        my_score = 0
        if isinstance(user_status.get('score'), (int, float)):
            my_score = int(user_status.get('score', 0))

        mal_score = 0
        if isinstance(anime_details.get('mean'), (int, float)):
            mal_score = round(float(anime_details.get('mean', 0)), 2)

        # Extract episode information
        episodes_watched = 0
        if isinstance(user_status.get('num_episodes_watched'), (int, float)):
            episodes_watched = int(user_status.get('num_episodes_watched', 0))

        total_episodes = 0
        if isinstance(anime_details.get('num_episodes'), (int, float)):
            total_episodes = int(anime_details.get('num_episodes', 0))

        # Extract rank
        rank = 0
        if isinstance(anime_details.get('rank'), (int, float)):
            rank = int(anime_details.get('rank', 0))

        # Create progress string
        total_ep_str = str(total_episodes) if total_episodes > 0 else "?"
        progress = f"{episodes_watched}/{total_ep_str}"

        # Calculate completion percentage
        completion_percentage = 0
        if total_episodes > 0:
            completion_percentage = round((episodes_watched / total_episodes) * 100, 1)

        # Extract alternative titles
        alternative_titles = anime_details.get('alternative_titles', {})
        alt_title = ""
        if isinstance(alternative_titles, dict):
            # Prefer English title, then Japanese, then synonyms
            if 'en' in alternative_titles and alternative_titles['en']:
                alt_title = alternative_titles['en'][0] if isinstance(alternative_titles['en'], list) else str(alternative_titles['en'])
            elif 'ja' in alternative_titles and alternative_titles['ja']:
                alt_title = alternative_titles['ja'][0] if isinstance(alternative_titles['ja'], list) else str(alternative_titles['ja'])
            elif 'synonyms' in alternative_titles and alternative_titles['synonyms']:
                alt_title = alternative_titles['synonyms'][0] if isinstance(alternative_titles['synonyms'], list) else str(alternative_titles['synonyms'])

        # Extract studios
        studios = anime_details.get('studios', [])
        studio_names = []
        if isinstance(studios, list):
            for studio in studios:
                if isinstance(studio, dict) and 'name' in studio:
                    studio_names.append(studio['name'])

        studios_str = ", ".join(studio_names) if studio_names else "Unknown Studio"

        # Get streaming platforms if country is specified
        streaming_info = "Country not specified"
        if country:
            streaming_info = get_streaming_platforms_for_anime(anime_details, country)

        return {
            'title': anime_details.get('title', 'Unknown Title'),
            'alternative_title': alt_title if alt_title else "No alternative title",
            'studios': studios_str,
            'image_url': image_url,
            'my_score': my_score,
            'mal_score': mal_score,
            'episodes_watched': episodes_watched,
            'total_episodes': total_episodes,
            'rank': rank,
            'status': user_status.get('status', 'unknown'),
            'progress': progress,
            'completion_percentage': completion_percentage,
            'media_type': anime_details.get('media_type', 'unknown'),
            'anime_status': anime_details.get('status', 'unknown'),
            'genres': anime_details.get('genres', []),
            'rating': anime_details.get('rating', 'unknown'),
            'streaming_platforms': streaming_info
        }
    except Exception:
        return None


def get_complete_anime_data(status):
    """
    Gets complete anime data by making individual requests for each anime.

    Args:
        status (str): Anime list status

    Returns:
        list: List of processed anime data
    """
    anime_ids, error = get_anime_list_ids(status)
    if error or not anime_ids:
        return None

    complete_data = []
    for anime_id in anime_ids:
        anime_details = get_individual_anime_details(anime_id)
        if not anime_details:
            continue

        user_status = get_user_anime_status(anime_id, status)
        processed_anime = process_individual_anime(anime_details, user_status)
        if processed_anime:
            complete_data.append(processed_anime)

    return complete_data


def sort_anime_dataframe(df, sort_parameter, ascending=False):
    """
    Sorts the anime DataFrame based on the specified parameter.

    Args:
        df (pd.DataFrame): DataFrame to sort
        sort_parameter (str): Column to sort by
        ascending (bool): Sort order

    Returns:
        pd.DataFrame: Sorted DataFrame
    """
    if sort_parameter not in df.columns:
        return df

    # Ensure numeric columns are properly typed
    numeric_columns = ['my_score', 'mal_score', 'episodes_watched', 'total_episodes', 'rank', 'completion_percentage']
    if sort_parameter in numeric_columns:
        df[sort_parameter] = pd.to_numeric(df[sort_parameter], errors='coerce').fillna(0)

    if sort_parameter == 'rank':
        # For rank: lower rank number = better ranking
        df_ranked = df[df[sort_parameter] > 0].copy()
        df_unranked = df[df[sort_parameter] == 0].copy()

        if not df_ranked.empty:
            df_ranked = df_ranked.sort_values(sort_parameter, ascending=True)

        df_sorted = pd.concat([df_ranked, df_unranked], ignore_index=True)

    elif sort_parameter in ['my_score', 'mal_score']:
        # For scores: handle 0 values (unscored) properly
        df_scored = df[df[sort_parameter] > 0].copy()
        df_unscored = df[df[sort_parameter] == 0].copy()

        if not df_scored.empty:
            df_scored = df_scored.sort_values(sort_parameter, ascending=ascending)

        df_sorted = pd.concat([df_scored, df_unscored], ignore_index=True)

    elif sort_parameter in ['title', 'alternative_title', 'studios']:
        # For text fields: alphabetical sorting
        df_sorted = df.sort_values(sort_parameter, ascending=ascending, key=lambda x: x.str.lower())

    else:
        # For other parameters: standard sorting
        df_sorted = df.sort_values(sort_parameter, ascending=ascending)

    return df_sorted.reset_index(drop=True)


def create_anime_table(anime_data, sort_parameter, ascending=False):
    """
    Creates a sorted table with anime data including streaming platforms.

    Args:
        anime_data (list): List of anime data dictionaries
        sort_parameter (str): Column to sort by
        ascending (bool): Sort order

    Returns:
        pd.DataFrame: Formatted table ready for display
    """
    if not anime_data:
        return pd.DataFrame()

    try:
        df = pd.DataFrame(anime_data)
    except Exception:
        return pd.DataFrame()

    if sort_parameter not in df.columns:
        return pd.DataFrame()

    df_sorted = sort_anime_dataframe(df, sort_parameter, ascending)
    table_data = []

    for idx, row in df_sorted.iterrows():
        try:
            # Create image HTML
            if row["image_url"] and str(row["image_url"]).strip():
                image_html = (
                    f'<img src="{row["image_url"]}" width="60" height="80" '
                    f'style="object-fit: cover; border-radius: 4px;" '
                    f'onerror="this.style.display=\'none\'; this.nextElementSibling.style.display=\'flex\';" '
                    f'alt="{row["title"]}">'
                    f'<div style="width:60px;height:80px;background:#f0f0f0;display:none;align-items:center;'
                    f'justify-content:center;font-size:10px;color:#999;border-radius:4px;text-align:center;">'
                    f'No Image</div>'
                )
            else:
                image_html = (
                    '<div style="width:60px;height:80px;background:#f0f0f0;display:flex;align-items:center;'
                    'justify-content:center;font-size:10px;color:#999;border-radius:4px;text-align:center;">'
                    'No Image</div>'
                )

            # Format sort parameter value
            sort_value = row[sort_parameter]
            if sort_parameter == 'rank':
                sort_value = f"#{int(sort_value)}" if sort_value > 0 else "Unranked"
            elif sort_parameter in ['my_score', 'mal_score']:
                sort_value = f"{sort_value}/10" if sort_value > 0 else "Not scored"
            elif sort_parameter == 'progress':
                sort_value = str(row['progress'])
            elif sort_parameter == 'completion_percentage':
                sort_value = f"{sort_value}%" if sort_value > 0 else "0%"
            else:
                sort_value = str(sort_value)

            # Determine additional column value
            additional_value = ""
            if sort_parameter not in ['rank', 'my_score', 'mal_score']:
                if row['rank'] > 0:
                    additional_value = f"Rank: #{int(row['rank'])}"
                elif row['mal_score'] > 0:
                    additional_value = f"MAL: {row['mal_score']}/10"
                elif row['my_score'] > 0:
                    additional_value = f"My Score: {int(row['my_score'])}/10"
                else:
                    additional_value = "No rating"
            elif sort_parameter == 'rank':
                if row['mal_score'] > 0:
                    additional_value = f"MAL: {row['mal_score']}/10"
                elif row['my_score'] > 0:
                    additional_value = f"My Score: {int(row['my_score'])}/10"
                else:
                    additional_value = "No score"
            elif sort_parameter in ['my_score', 'mal_score']:
                if row['rank'] > 0:
                    additional_value = f"Rank: #{int(row['rank'])}"
                else:
                    other_score = 'mal_score' if sort_parameter == 'my_score' else 'my_score'
                    if row[other_score] > 0:
                        score_name = "MAL" if other_score == 'mal_score' else "My Score"
                        score_val = row[other_score] if other_score == 'mal_score' else int(row[other_score])
                        additional_value = f"{score_name}: {score_val}/10"
                    else:
                        additional_value = "No other rating"

            streaming_platforms = str(row.get('streaming_platforms', 'Not checked'))

            table_row = {
                'Order': idx + 1,
                'Title': str(row['title']),
                'Alternative Title': str(row['alternative_title']),
                'Studio': str(row['studios']),
                'Image': image_html,
                f'{sort_parameter.replace("_", " ").title()}': sort_value,
                'Additional Info': additional_value,
                'Streaming Platforms': streaming_platforms
            }

            table_data.append(table_row)

        except Exception:
            continue

    if not table_data:
        return pd.DataFrame()

    return pd.DataFrame(table_data)


def display_anime_table_interactive():
    """
    Interactive function to display anime table with user choices.
    Uses tokens['access_token'] and COUNTRY variables automatically.

    Returns:
        pd.DataFrame: Generated table or None if error
    """
    print("🎌 MyAnimeList Table Generator - Interactive Mode")
    print("=" * 50)

    # Check if tokens are available
    try:
        oauth2_token = tokens['access_token']
    except (NameError, KeyError):
        print("❌ OAuth2 tokens not available. Please authenticate first.")
        return None

    # Check if COUNTRY is available
    try:
        country = COUNTRY
        print(f"🌍 Streaming availability will be checked for: {country}")
    except NameError:
        print("⚠️  COUNTRY variable not defined. Streaming info will be skipped.")

    # Status options
    status_options = {
        '1': ('watching', 'Currently Watching'),
        '2': ('completed', 'Completed'),
        '3': ('on_hold', 'On Hold'),
        '4': ('dropped', 'Dropped'),
        '5': ('plan_to_watch', 'Plan to Watch')
    }

    print("📋 Available anime list statuses:")
    for key, (value, display) in status_options.items():
        print(f"  {key}. {display}")

    # Get status choice
    while True:
        status_choice = input("\nSelect status (1-5): ").strip()
        if status_choice in status_options:
            selected_status = status_options[status_choice][0]
            break
        print("❌ Invalid choice. Please select 1-5.")

    print(f"✅ Selected status: {status_options[status_choice][1]}")

    # Sort parameter options
    sort_options = {
        '1': ('my_score', 'My Score'),
        '2': ('mal_score', 'MAL Score'),
        '3': ('rank', 'MAL Rank'),
        '4': ('episodes_watched', 'Episodes Watched'),
        '5': ('total_episodes', 'Total Episodes'),
        '6': ('progress', 'Progress'),
        '7': ('completion_percentage', 'Completion %'),
        '8': ('title', 'Title'),
        '9': ('alternative_title', 'Alternative Title'),
        '10': ('studios', 'Studio')
    }

    print("\n📊 Available sorting parameters:")
    for key, (value, display) in sort_options.items():
        print(f"  {key}. {display}")

    # Get sort choice
    while True:
        sort_choice = input("\nSelect sorting parameter (1-10): ").strip()
        if sort_choice in sort_options:
            selected_sort = sort_options[sort_choice][0]
            break
        print("❌ Invalid choice. Please select 1-10.")

    print(f"✅ Selected sorting: {sort_options[sort_choice][1]}")

    # Get sort order
    order_choice = input("\nSort order - Ascending (a) or Descending (d)? [d]: ").strip().lower()
    ascending = True if order_choice == 'a' else False

    print(f"✅ Sort order: {'Ascending' if ascending else 'Descending'}")

    return _generate_table(selected_status, selected_sort, ascending)


def display_anime_table_preset(status='watching', sort_param='my_score', ascending=False):
    """
    Non-interactive version with preset parameters.
    Uses tokens['access_token'] and COUNTRY variables automatically.

    Args:
        status (str): Anime list status
        sort_param (str): Parameter to sort by
        ascending (bool): Sort order

    Returns:
        pd.DataFrame: Generated table or None if error
    """
    print("🎌 MyAnimeList Table Generator - Preset Mode")
    print(f"📋 Status: {status.replace('_', ' ').title()}")
    print(f"📊 Sort by: {sort_param.replace('_', ' ').title()}")
    print(f"📈 Order: {'Ascending' if ascending else 'Descending'}")

    # Check if tokens are available
    try:
        oauth2_token = tokens['access_token']
    except (NameError, KeyError):
        print("❌ OAuth2 tokens not available. Please authenticate first.")
        return None

    # Check if COUNTRY is available
    try:
        country = COUNTRY
        print(f"🌍 Streaming availability will be checked for: {country}")
    except NameError:
        print("⚠️  COUNTRY variable not defined. Streaming info will be skipped.")

    print("=" * 50)

    return _generate_table(status, sort_param, ascending)


def _generate_table(status, sort_param, ascending):
    """
    Internal function to generate the table using individual anime requests.
    Uses tokens['access_token'] and COUNTRY variables automatically.

    Args:
        status (str): Anime list status
        sort_param (str): Parameter to sort by
        ascending (bool): Sort order

    Returns:
        pd.DataFrame: Generated table or None if error
    """
    # Check if tokens are available
    try:
        oauth2_token = tokens['access_token']
    except (NameError, KeyError):
        print("❌ OAuth2 tokens not available. Please authenticate first.")
        return None

    print("🔄 Fetching complete anime data with individual requests...")

    anime_data = get_complete_anime_data(status)

    if not anime_data:
        print("❌ No anime data could be retrieved")
        return None

    print("🔄 Creating and sorting table...")
    final_table = create_anime_table(anime_data, sort_param, ascending)

    if final_table.empty:
        print("❌ No table data could be created")
        return None

    print(f"\n🎉 Table created successfully!")
    print(f"📊 Sorted by: {sort_param.replace('_', ' ').title()}")
    print(f"📈 Order: {'Ascending' if ascending else 'Descending'}")
    print(f"📝 Total entries: {len(final_table)}")

    # Enhanced CSS for better table appearance
    table_css = """
    <style>
    #anime_table {
        border-collapse: collapse;
        margin: 20px 0;
        font-size: 14px;
        font-family: Arial, sans-serif;
        min-width: 100%;
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.15);
    }
    #anime_table thead tr {
        background-color: #009879;
        color: #ffffff;
        text-align: left;
    }
    #anime_table th,
    #anime_table td {
        padding: 12px 15px;
        border: 1px solid #dddddd;
        vertical-align: top;
    }
    #anime_table tbody tr {
        border-bottom: 1px solid #dddddd;
    }
    #anime_table tbody tr:nth-of-type(even) {
        background-color: #f3f3f3;
    }
    #anime_table tbody tr:hover {
        background-color: #f1f1f1;
    }
    #anime_table td:nth-child(5) {
        text-align: center;
    }
    #anime_table td:nth-child(8) {
        max-width: 200px;
        word-wrap: break-word;
    }
    #anime_table td:nth-child(3),
    #anime_table td:nth-child(4) {
        max-width: 150px;
        word-wrap: break-word;
    }
    </style>
    """

    # Display table with improved formatting
    html_table = table_css + final_table.to_html(escape=False, index=False, table_id="anime_table")
    display(HTML(html_table))

    return final_table


## 🚀 Function Execution Guide

This section explains how to generate your anime table once you're authenticated.

---

### 🎮 Interactive Mode

The **interactive mode** walks you through the process step-by-step via prompts.

#### 🧪 Example Call:

`anime_table = display_anime_table_interactive()`


#### 🧭 What to Expect:

1. **Select Anime List Status**
   - Available options:
     - `Currently Watching`
     - `Completed`
     - `On Hold`
     - `Dropped`
     - `Plan to Watch`
   - Example input: `2` → ✅ Selected: `Completed`

2. **Choose Sorting Parameter**
   - Options include:
     - `My Score`
     - `MAL Score`
     - `MAL Rank`
     - `Episodes Watched`
     - `Total Episodes`
     - `Progress`
     - `Completion %`
     - `Title`
   - Example input: `1` → ✅ Selected: `My Score`

3. **Set Sort Order**
   - Input `a` for ascending or `d` for descending.
   - Example: `d` → ✅ Order: `Descending`

4. **Execution**
   - The system will:
     - Fetch anime data from MyAnimeList
     - Use JustWatch to check streaming availability
     - Generate and display a styled table

---

### ⚡ Preset Mode

The **preset mode** lets you skip prompts and run the function with predefined values — great for scripting.


#### 💼 Examples:

**Top "Plan to Watch" by MAL Score:**

`anime_table = display_anime_table_preset(status='plan_to_watch', sort_param='mal_score', ascending=False)`

**Top Completed by Personal Rating:**

`anime_table = display_anime_table_preset( status='completed', sort_param='my_score', ascending=False)`

**Currently Watching by Progress:**

`anime_table = display_anime_table_preset(status='watching', sort_param='completion_percentage', ascending=True)`

---

### 📊 Parameter Reference

#### 📁 Status Options

| Value            | Meaning                    |
|------------------|----------------------------|
| `'watching'`     | Currently watching         |
| `'completed'`    | Finished anime             |
| `'on_hold'`      | Temporarily paused         |
| `'dropped'`      | Stopped watching           |
| `'plan_to_watch'`| Planning to watch          |

#### 📈 Sorting Options

| Parameter              | Description             | Ideal Use                         |
|------------------------|-------------------------|------------------------------------|
| `'my_score'`           | Your personal rating    | Find your personal favorites       |
| `'mal_score'`          | Community rating        | Discover popular picks             |
| `'rank'`               | Global ranking          | Identify top-ranked titles         |
| `'episodes_watched'`   | Episodes watched        | Track progress                     |
| `'completion_percentage'` | Viewing completion % | Spot nearly finished shows         |
| `'title'`              | Alphabetical title      | Browse your list systematically    |

#### 🌍 Country Codes

| Country         | Code | Country     | Code |
|-----------------|------|-------------|------|
| United States   | `US` | Spain       | `ES` |
| United Kingdom  | `GB` | Germany     | `DE` |
| France          | `FR` | Japan       | `JP` |
| Canada          | `CA` | Australia   | `AU` |



In [59]:
display_anime_table_interactive()

🎌 MyAnimeList Table Generator - Interactive Mode
🌍 Streaming availability will be checked for: ES
📋 Available anime list statuses:
  1. Currently Watching
  2. Completed
  3. On Hold
  4. Dropped
  5. Plan to Watch
✅ Selected status: Currently Watching

📊 Available sorting parameters:
  1. My Score
  2. MAL Score
  3. MAL Rank
  4. Episodes Watched
  5. Total Episodes
  6. Progress
  7. Completion %
  8. Title
  9. Alternative Title
  10. Studio
✅ Selected sorting: MAL Score
✅ Sort order: Descending
🔄 Fetching complete anime data with individual requests...
🔄 Creating and sorting table...

🎉 Table created successfully!
📊 Sorted by: Mal Score
📈 Order: Descending
📝 Total entries: 21


Order,Title,Alternative Title,Studio,Image,Mal Score,Additional Info,Streaming Platforms
1,Kaguya-sama wa Kokurasetai: First Kiss wa Owaranai,Kaguya-sama: Love is War -The First Kiss That Never Ends-,A-1 Pictures,No Image,8.74/10,Rank: #48,Not available in this country
2,One Piece,One Piece,Toei Animation,No Image,8.73/10,Rank: #50,"Crunchyroll, Crunchyroll Amazon Channel"
3,Ore dake Level Up na Ken Season 2: Arise from the Shadow,Solo Leveling Season 2: Arise from the Shadow,A-1 Pictures,No Image,8.72/10,Rank: #54,"Crunchyroll, Crunchyroll Amazon Channel"
4,Dungeon Meshi,Delicious in Dungeon,Trigger,No Image,8.6/10,Rank: #98,"Netflix, Netflix Standard with Ads"
5,[Oshi no Ko],[Oshi No Ko],Doga Kobo,No Image,8.57/10,Rank: #107,"Netflix, Netflix Standard with Ads, AMC+ Amazon Channel, AMC Plus Apple TV Channel"
6,Tu Bian Yingxiong X,To Be Hero X,"Pb Animation Co. Ltd., LAN Studio, Paper Plane Animation Studio",No Image,8.54/10,Rank: #126,"Crunchyroll, Crunchyroll Amazon Channel"
7,Ousama Ranking,Ranking of Kings,Wit Studio,No Image,8.49/10,Rank: #149,"Crunchyroll, Crunchyroll Amazon Channel"
8,Summertime Render,Summer Time Rendering,OLM,No Image,8.47/10,Rank: #152,Disney Plus
9,Kono Subarashii Sekai ni Shukufuku wo! 3,KonoSuba: God's Blessing on This Wonderful World! 3,Drive,No Image,8.35/10,Rank: #242,"Netflix, Netflix Standard with Ads, Crunchyroll Amazon Channel"
10,Tengoku Daimakyou,Heavenly Delusion,Production I.G,No Image,8.21/10,Rank: #381,Disney Plus


,Order,Title,Alternative Title,Studio,Image,Mal Score,Additional Info,Streaming Platforms
0,1,Kaguya-sama wa Kokurasetai: First Kiss wa Owar...,Kaguya-sama: Love is War -The First Kiss That ...,A-1 Pictures,"<img src=""https://cdn.myanimelist.net/images/a...",8.74/10,Rank: #48,Not available in this country
1,2,One Piece,One Piece,Toei Animation,"<img src=""https://cdn.myanimelist.net/images/a...",8.73/10,Rank: #50,"Crunchyroll, Crunchyroll Amazon Channel"
2,3,Ore dake Level Up na Ken Season 2: Arise from ...,Solo Leveling Season 2: Arise from the Shadow,A-1 Pictures,"<img src=""https://cdn.myanimelist.net/images/a...",8.72/10,Rank: #54,"Crunchyroll, Crunchyroll Amazon Channel"
3,4,Dungeon Meshi,Delicious in Dungeon,Trigger,"<img src=""https://cdn.myanimelist.net/images/a...",8.6/10,Rank: #98,"Netflix, Netflix Standard with Ads"
4,5,[Oshi no Ko],[Oshi No Ko],Doga Kobo,"<img src=""https://cdn.myanimelist.net/images/a...",8.57/10,Rank: #107,"Netflix, Netflix Standard with Ads, AMC+ Amazo..."
5,6,Tu Bian Yingxiong X,To Be Hero X,"Pb Animation Co. Ltd., LAN Studio, Paper Plane...","<img src=""https://cdn.myanimelist.net/images/a...",8.54/10,Rank: #126,"Crunchyroll, Crunchyroll Amazon Channel"
6,7,Ousama Ranking,Ranking of Kings,Wit Studio,"<img src=""https://cdn.myanimelist.net/images/a...",8.49/10,Rank: #149,"Crunchyroll, Crunchyroll Amazon Channel"
7,8,Summertime Render,Summer Time Rendering,OLM,"<img src=""https://cdn.myanimelist.net/images/a...",8.47/10,Rank: #152,Disney Plus
8,9,Kono Subarashii Sekai ni Shukufuku wo! 3,KonoSuba: God's Blessing on This Wonderful Wor...,Drive,"<img src=""https://cdn.myanimelist.net/images/a...",8.35/10,Rank: #242,"Netflix, Netflix Standard with Ads, Crunchyrol..."
9,10,Tengoku Daimakyou,Heavenly Delusion,Production I.G,"<img src=""https://cdn.myanimelist.net/images/a...",8.21/10,Rank: #381,Disney Plus
